In [2]:
import torch 
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as f
from torch.utils.data import DataLoader
import torchvision.datasets as datasets
import torchvision.transforms as transforms

In [3]:
class NN(nn.Module):
    def __init__(self, input_size, num_columns):
        super(NN, self).__init__()
        self.fc1 = nn.Linear(input_size, 50)
        self.fc2 = nn.Linear(50, num_columns)
    
    def forward(self, x):
        x = f.relu(self.fc1(x))
        x = self.fc2(x)
        return x

In [4]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [5]:
input_size = 784
num_classes = 10
learning_rate = 0.001
num_epochs = 1
batch_size = 64

In [6]:
train_dataset = datasets.MNIST(root='dataset/', train=True, transform=transforms.ToTensor(), download=True)
train_dataloader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
test_dataset = datasets.MNIST(root='dataset/', train=False, transform=transforms.ToTensor(), download=True)
test_dataloader = DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=True)

In [7]:
model = NN(input_size=input_size, num_columns=num_classes).to(device)

In [8]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), learning_rate)

In [ ]:
model.train()
for epochs in range(num_epochs):
    for batch_size, (data, targets) in enumerate(train_dataloader):
        data = data.to(device)
        targets = targets.to(device)
        